# PyTorch Geometric基础知识
PyG 是一款号称比 DGL 快 14 倍的基于 PyTorch 的几何深度学习框架，可以简单方便的实现图神经网络。

学习PyG的实践代码. [学习资料](https://www.pytorchtutorial.com/pytorch-geometric-for-gnn/#PyTorch_Geometric)

在源教程的基础上稍作改动, 算是更加适合我的版本吧.

In [60]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU

from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm


print(torch.__version__)
print(torch.cuda.is_available())

1.5.1
True


## Data类
torch_geometric.data 包里有一个 Data 类，通过 Data 类我们可以很方便的创建图结构。

定义一个图结构，需要以下变量：

+ 每个节点（node）的 features
+ 边的连接关系或者边的 features

### Example
表示下面这张图: 一共有四个节点 v1,v2,v3,v4，其中每个节点都有一个二维的特征向量x和一个标签 y。
>20/7/1 猜测x是feature, y是label

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14b8d64bfa4.png)

1. 表示节点的 features

In [2]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

2. 表示边的连接(这里边没有feature)

>通过一组列向量来描述边的连接.

In [3]:
edge_index = torch.tensor([[0,0,1,2,3],
                           [1,3,0,1,2]], dtype=torch.long)

3. 定义图结构

In [4]:
data = Data(x=x, y=y, edge_index=edge_index)

## Dataset
PyG 里有两种数据集类型：InMemoryDataset 和 Dataset，第一种适用于可以全部放进内存中的小数据集，第二种则适用于不能一次性放进内存中的大数据集。我们以 InMemoryDataset 为例。
> 20/7/1 猜测两种Dataset的API一致

InMemoryDataset 中有下列四个函数需要我们实现：

`raw_file_names()`
返回一个包含所有未处理过的数据文件的文件名的列表。

起始也可以返回一个空列表，然后在后面要说的 process() 函数里再定义。

`processed_file_names()`
返回一个包含所有处理过的数据文件的文件名的列表。

`download()`
如果在数据加载前需要先下载，则在这里定义下载过程，下载到 self.raw_dir 中定义的文件夹位置。

如果不需要下载，返回 pass 即可。

`process()`
这是最重要的一个函数，我们需要在这个函数里把数据处理成一个 Data 对象。

下面是官方的一个示例代码：

In [5]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
 
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
 
    @property
    def processed_file_names(self):
        return ['data.pt']
 
    def download(self):
        # Download to `self.raw_dir`.
        pass
 
    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]
 
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
 
        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
 
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

本文接下来会介绍如何用 RecSys Challenge 2015 的数据创建一个自定义数据集。

## DataLoader
就是使用原始Pytorch中的Dataloader

## MessagePassing
Message Passing 是图网络中学习 node embedding 的重要方法。

Message Passing 的公示如下：

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14c0ce52bab.png)

其中，x 表示表格节点的 embedding，e 表示边的特征，ϕ 表示 message 函数，□ 表示聚合 aggregation 函数，γ 表示 update 函数。上标表示层的 index，比如说，当 k = 1 时，x 则表示所有输入网络的图结构的数据。

下面是每个函数的介绍：

`propagate(edge_index, size=None, **kwargs)`
这个函数最终会调用 message 和 update 函数。

`message(**kwargs)`
这个函数定义了对于每个节点对 (xi,xj)，怎样生成信息（message）。

`update(aggr_out, **kwargs)`
这个函数利用聚合好的信息（message）更新每个节点的 embedding。

### 示例：SageConv
我们来看看怎样实现论文 “Inductive Representation Learning on Large Graphs” 中的 SageConv 层。SageConv 的 Message Passing 定义如下：

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14c3404305b.png)

聚合函数（aggregation）我们用最大池化（max pooling），这样上述公示中的 AGGREGATE 可以写为：

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14c39149141.png)

上述公式中，对于每个邻居节点，都和一个 weighted matrix 相乘，并且加上一个 bias，传给一个激活函数。相关代码如下：


In [6]:
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max')
        # for message
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        # for update
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]
 
        x_j = self.lin(x_j)
        x_j = self.act(x_j)
      
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]

        new_embedding = torch.cat([aggr_out, x], dim=1)
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)

        return new_embedding

# 示例：RecSys Challenge 2015
RecSys Challenge 2015 是一个挑战赛，主要目的是创建一个 session-based recommender system。主要任务有两个：

1. 预测经过一系列的点击后，是否会产生购买行为。
2. 预测购买的商品。

数据下载地址在[这里](https://www.pytorchtutorial.com/goto/https://2015.recsyschallenge.com/challenge.html)。数据主要包含两部分：yoochoose-clicks.dat 点击数据, 和 yoochoose-buys.dat 购买行为数据。

## 数据预处理
LabelEncoder作用：给标签分配一个0 ~ n_classes-1之间的编码。

In [7]:
df = pd.read_csv('data/yoochoose-data/yoochoose-clicks.dat', header=None)
df.columns=['session_id','timestamp','item_id','category']
 
buy_df = pd.read_csv('data/yoochoose-data/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']

item_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)

df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,2053,0
1,1,2014-04-07T10:54:09.868Z,2052,0
2,1,2014-04-07T10:54:46.998Z,2054,0
3,1,2014-04-07T10:57:00.306Z,9876,0
4,2,2014-04-07T13:56:37.614Z,19448,0


对数据随机抽样（数据太多了）

In [8]:
sampled_session_id = np.random.choice(df.session_id.unique(), 1000000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    1000000
timestamp     3567939
item_id         35571
category          243
dtype: int64

给Data获取标签。标签是是否购买（True、False），如果这个session_id在buy_df中就是买了。

In [9]:
buy_df.head()

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


In [10]:
df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
106,34,2014-04-07T14:29:57.726Z,29383,0,False
107,34,2014-04-07T14:33:22.860Z,41232,0,False
133,37,2014-04-04T16:26:07.882Z,41203,0,False
134,37,2014-04-04T16:26:43.356Z,2425,0,False
168,48,2014-04-01T12:12:08.080Z,24301,0,False


## 创建Dataset
一个session中的每一个商品（item）看作一个节点，每个session中的所有商品组成一个图

首先，我们将数据集按照 session_id 进行分组，分组过程中 item_id 也要被重新编码，因为对于每个图，每个节点的 index 应该从 0 开始：

>ps: tqdm库，进度条迭代器

>collate()函数将列表转换为一个torch_geometric.data.Data对象。处理后的数据被整合到了一个数据对象中（作为返回值），同时返回一个slices字典来获取到这个数据对象中单个数据

In [12]:
a = df.groupby('session_id')

In [36]:
for session_id, group in a:
    if group.shape[0] < 10:
        continue
    print(session_id)
    break
# group.reset_index(drop = True)
sess_item_id = LabelEncoder().fit_transform(group.item_id)
group = group.reset_index(drop=True)
group['sess_item_id'] = sess_item_id
group.label.values[0]


86


False

In [48]:
class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, df, root, transform=None, pre_transform=None):
        self.df = df
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['yoochoose_click_binary_1M_sess.dataset']
    
    def download(self):
        pass

    """
    将数据集包装成Data图对象
    """
    def process(self):
        data_list = []
    
        # process by session_id
        grouped = self.df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            # item_id 重新编码
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            
            # node_features 就是去重的 item_id，size为(n_item_id, 1)
            # 每一个item就是一个节点，所以要去重
            # 节点的x就是对应的全局item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']]\
                .sort_values('sess_item_id')\
                .item_id.drop_duplicates().values
            node_features = torch.LongTensor(node_features).unsqueeze(1)
            
            # 生成边，如果一个session依次点击了 1，2，3，2
            # 则有三条边 1->2, 2->3, 3->2
            # 数据集就是按照时间戳排序的，所以可以像下面这样直接用
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]
            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

            # 指明 x, y, edge_index
            x = node_features
            # 原教程的 y 貌似有点问题
            y = torch.FloatTensor([group.label.values[0]])
#             y = torch.FloatTensor(group.label.values)
            
            data = Data(x=x, y=y, edge_index=edge_index)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
        
# 实例化
dataset = YooChooseBinaryDataset(df, "./data/yoochoose-data/")

Processing...


100%|██████████| 1000000/1000000 [1:22:08<00:00, 202.92it/s]


Done!


对数据集随机排序，分割

In [50]:
dataset = dataset.shuffle()
train_dataset = dataset[:800000]
val_dataset = dataset[800000:900000]
test_dataset = dataset[900000:]
len(train_dataset), len(val_dataset), len(test_dataset)

(800000, 100000, 100000)

## 创建GNN

In [80]:
embed_dim = 128
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        
 
        x = F.relu(self.conv1(x, edge_index))
 
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = F.relu(self.conv3(x, edge_index))
 
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = x1 + x2 + x3
 
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
 
        x = torch.sigmoid(self.lin3(x)).squeeze(1)
 
        return x

## 训练
训练过程中，我们使用 Adam 优化器，学习率 0.005，损失函数是 BCE：

In [87]:
def train():
    model.train()
    print("Training...")
    loss_all = 0
    for data in tqdm(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)   

In [82]:
# super params
batch_size = 256
num_epochs = 10

In [83]:
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=batch_size)

## 评估
这个数据集非常的不平衡，因为大多数的 session 里没有购买行为。也就是说，如果一个模型将所有的结果都预测为 false，也能达到 90% 的准确率。因此，这里我们不使用 accuracy 作为评测标准，而是使用 Area Under Curve (AUC)：

In [88]:
def evaluate(loader):
    model.eval()
    
    predictions = []
    labels = []
 
    with torch.no_grad():
        for data in tqdm(loader):
 
            data = data.to(device)
            pred = model(data).detach().cpu().numpy()
 
            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

## 开始训练

In [89]:
for epoch in range(num_epochs):
    loss = train()
    print("Train done. Evaluating")
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

  0%|          | 1/3125 [00:00<09:11,  5.66it/s]

Training... 

  0%|          | 1/3125 [00:00<07:27,  6.98it/s]

Train done. Evaluating


100%|██████████| 3125/3125 [07:46<00:00,  6.70it/s]


NameError: name 'val_loader' is not defined

In [92]:
print("Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}".format(epoch, loss, train_acc))

TypeError: unsupported format string passed to NoneType.__format__